In [1]:
import pandas as pd
import numpy as np

In [2]:
import pandas.util.testing as tm; tm.N = 3
def unpivot(frame):
    N, K = frame.shape
    data = {'value' : frame.values.ravel('F'),
            'variable' : np.asarray(frame.columns).repeat(N),
            'date' : np.tile(np.asarray(frame.index), K)}
    return pd.DataFrame(data, columns=['date', 'variable', 'value'])
df = unpivot(tm.makeTimeDataFrame())

In [3]:
df

,date,variable,value
0,2000-01-03,A,-0.350710
1,2000-01-04,A,-0.492709
2,2000-01-05,A,0.179595
3,2000-01-03,B,-1.139879
4,2000-01-04,B,0.700122
5,2000-01-05,B,1.773256
6,2000-01-03,C,0.820958
7,2000-01-04,C,-0.606836
8,2000-01-05,C,0.078485
9,2000-01-03,D,-1.679623


In [6]:
df.pivot(index='date',values='value',columns='variable')

variable,A,B,C,D
date,,,,
2000-01-03,-0.350710,-1.139879,0.820958,-1.679623
2000-01-04,-0.492709,0.700122,-0.606836,0.682431
2000-01-05,0.179595,1.773256,0.078485,0.486778


In [11]:
df['value2'] = df['value'] * 2
p = df.pivot('date','variable')
p['value']

variable,A,B,C,D
date,,,,
2000-01-03,-0.350710,-1.139879,0.820958,-1.679623
2000-01-04,-0.492709,0.700122,-0.606836,0.682431
2000-01-05,0.179595,1.773256,0.078485,0.486778


In [ ]:
# pivot changes the value into columns

In [12]:
# stack and unstack are designed for multiIndex
indx = pd.MultiIndex.from_product([['bar','baz','foo','qux'],['one','two']],names=['first', 'second'])

In [13]:
df = pd.DataFrame(np.random.randn(8, 2), index=indx, columns=['A', 'B'])

In [14]:
df

A         B
first second                    
bar   one     0.752752 -0.696195
      two    -2.783731 -0.662233
baz   one    -1.406770 -0.883630
      two     0.555568 -2.065790
foo   one     0.098166  0.106246
      two     0.294153  0.978375
qux   one     1.291555  0.350292
      two    -0.084064  0.851005

In [40]:
df.unstack()


A                   B          
second       one       two       one       two
first                                         
bar     0.752752 -2.783731 -0.696195 -0.662233
baz    -1.406770  0.555568 -0.883630 -2.065790
foo     0.098166  0.294153  0.106246  0.978375
qux     1.291555 -0.084064  0.350292  0.851005

In [18]:
type(df.stack())

pandas.core.series.Series

In [39]:
df.stack().unstack(level=0)
# left most level 

first          bar       baz       foo       qux
second                                          
one    A  0.752752 -1.406770  0.098166  1.291555
       B -0.696195 -0.883630  0.106246  0.350292
two    A -2.783731  0.555568  0.294153 -0.084064
       B -0.662233 -2.065790  0.978375  0.851005

In [34]:
df.stack().unstack(level=1)

second        one       two
first                      
bar   A  0.752752 -2.783731
      B -0.696195 -0.662233
baz   A -1.406770  0.555568
      B -0.883630 -2.065790
foo   A  0.098166  0.294153
      B  0.106246  0.978375
qux   A  1.291555 -0.084064
      B  0.350292  0.851005

In [35]:
df.stack().unstack(level=2)

A         B
first second                    
bar   one     0.752752 -0.696195
      two    -2.783731 -0.662233
baz   one    -1.406770 -0.883630
      two     0.555568 -2.065790
foo   one     0.098166  0.106246
      two     0.294153  0.978375
qux   one     1.291555  0.350292
      two    -0.084064  0.851005

In [46]:
df.unstack().stack([1,0])

first  second   
bar    one     A    0.752752
               B   -0.696195
       two     A   -2.783731
               B   -0.662233
baz    one     A   -1.406770
               B   -0.883630
       two     A    0.555568
               B   -2.065790
foo    one     A    0.098166
               B    0.106246
       two     A    0.294153
               B    0.978375
qux    one     A    1.291555
               B    0.350292
       two     A   -0.084064
               B    0.851005
dtype: float64

In [ ]:
# stack is like the stack in the Memory making structure becomes an address and data
# unstack change the higher level address into some columns

In [48]:
df.unstack()

A                   B          
second       one       two       one       two
first                                         
bar     0.752752 -2.783731 -0.696195 -0.662233
baz    -1.406770  0.555568 -0.883630 -2.065790
foo     0.098166  0.294153  0.106246  0.978375
qux     1.291555 -0.084064  0.350292  0.851005

In [49]:
# melt change some of the columns' name to values, reverse of pivot
cheese = pd.DataFrame({'first' : ['John', 'Mary'],
                           'last' : ['Doe', 'Bo'],
                           'height' : [5.5, 6.0],
                           'weight' : [130, 150]})

In [50]:
cheese

,first,height,last,weight
0,John,5.5,Doe,130
1,Mary,6.0,Bo,150


In [66]:
che = cheese.melt(id_vars=['first','last']).set_index(['first','last'])
che.index = che.index.map(lambda x: ''.join(list(x))).rename('fullname')

In [71]:
che.sort_index().pivot(values='value',columns='variable')

variable,height,weight
fullname,,
JohnDoe,5.5,130.0
MaryBo,6.0,150.0


In [72]:
# wide_to_long looks weird

In [73]:
columns = pd.MultiIndex.from_tuples([('A', 'cat'), ('B', 'dog'),
                                         ('B', 'cat'), ('A', 'dog')],
                                        names=['exp', 'animal'])
    

index = pd.MultiIndex.from_product([('bar', 'baz', 'foo', 'qux'),
                                        ('one', 'two')],
                                       names=['first', 'second'])
    

df = pd.DataFrame(np.random.randn(8, 4), index=index, columns=columns)

In [74]:
df

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one    -0.214469  0.885186  0.107010  0.353281
      two     0.178076 -1.821079 -0.564283 -1.229566
baz   one     0.172431 -0.994584 -1.715162 -0.599263
      two     2.213867  0.163959 -0.388120 -0.312096
foo   one     0.754830 -1.758164 -0.322039  0.755660
      two     1.602840  0.896675  1.331704  0.993717
qux   one    -0.503406 -1.375622 -1.353653 -1.235032
      two     0.362915  0.263981  0.116362  1.434270

In [78]:
df.stack()

exp                         A         B
first second animal                    
bar   one    cat    -0.214469  0.107010
             dog     0.353281  0.885186
      two    cat     0.178076 -0.564283
             dog    -1.229566 -1.821079
baz   one    cat     0.172431 -1.715162
             dog    -0.599263 -0.994584
      two    cat     2.213867 -0.388120
             dog    -0.312096  0.163959
foo   one    cat     0.754830 -0.322039
             dog     0.755660 -1.758164
      two    cat     1.602840  1.331704
             dog     0.993717  0.896675
qux   one    cat    -0.503406 -1.353653
             dog    -1.235032 -1.375622
      two    cat     0.362915  0.116362
             dog     1.434270  0.263981

In [79]:
df.stack().mean(1)

first  second  animal
bar    one     cat      -0.053730
               dog       0.619234
       two     cat      -0.193104
               dog      -1.525322
baz    one     cat      -0.771366
               dog      -0.796924
       two     cat       0.912873
               dog      -0.074069
foo    one     cat       0.216395
               dog      -0.501252
       two     cat       1.467272
               dog       0.945196
qux    one     cat      -0.928529
               dog      -1.305327
       two     cat       0.239638
               dog       0.849126
dtype: float64

In [80]:
df.stack().mean(1).unstack()

animal             cat       dog
first second                    
bar   one    -0.053730  0.619234
      two    -0.193104 -1.525322
baz   one    -0.771366 -0.796924
      two     0.912873 -0.074069
foo   one     0.216395 -0.501252
      two     1.467272  0.945196
qux   one    -0.928529 -1.305327
      two     0.239638  0.849126

In [92]:
df.groupby(level=0, axis=0).mean()

exp            A         B                   A
animal       cat       dog       cat       dog
first                                         
bar    -0.018197 -0.467946 -0.228637 -0.438143
baz     1.193149 -0.415313 -1.051641 -0.455680
foo     1.178835 -0.430744  0.504832  0.874689
qux    -0.070246 -0.555821 -0.618645  0.099619

In [85]:
df

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one    -0.214469  0.885186  0.107010  0.353281
      two     0.178076 -1.821079 -0.564283 -1.229566
baz   one     0.172431 -0.994584 -1.715162 -0.599263
      two     2.213867  0.163959 -0.388120 -0.312096
foo   one     0.754830 -1.758164 -0.322039  0.755660
      two     1.602840  0.896675  1.331704  0.993717
qux   one    -0.503406 -1.375622 -1.353653 -1.235032
      two     0.362915  0.263981  0.116362  1.434270

In [96]:
df.stack().groupby(level=1).mean()

exp,A,B
second,,
one,-0.064496,-0.815878
two,0.655503,-0.000100


In [100]:
df.mean(axis=0,level=0)

exp            A         B                   A
animal       cat       dog       cat       dog
first                                         
bar    -0.018197 -0.467946 -0.228637 -0.438143
baz     1.193149 -0.415313 -1.051641 -0.455680
foo     1.178835 -0.430744  0.504832  0.874689
qux    -0.070246 -0.555821 -0.618645  0.099619

In [101]:
import datetime
df = pd.DataFrame({'A': ['one', 'one', 'two', 'three'] * 6,
                       'B': ['A', 'B', 'C'] * 8,
                       'C': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'] * 4,
                       'D': np.random.randn(24),
                       'E': np.random.randn(24),
                       'F': [datetime.datetime(2013, i, 1) for i in range(1, 13)] +
                            [datetime.datetime(2013, i, 15) for i in range(1, 13)]})

In [102]:
df

,A,B,C,D,E,F
0,one,A,foo,0.422497,0.922499,2013-01-01
1,one,B,foo,1.502079,0.046936,2013-02-01
2,two,C,foo,-0.725436,0.658554,2013-03-01
3,three,A,bar,-0.485136,-1.303791,2013-04-01
4,one,B,bar,-0.521530,-0.153625,2013-05-01
5,one,C,bar,0.087129,0.670994,2013-06-01
6,two,A,foo,-1.525045,-1.293172,2013-07-01
7,three,B,foo,-1.474063,-0.583320,2013-08-01
8,one,C,foo,-0.222699,-0.671653,2013-09-01
9,one,A,bar,0.943377,1.433455,2013-10-01


In [108]:
df[(df['A']=='one') & (df['C']=='foo') & (df['B']=='A')]['D']

0     0.422497
12    0.081403
Name: D, dtype: float64

In [110]:
df.pivot_table(values='D',columns='C',index=['A','B'])

C             bar       foo
A     B                    
one   A  0.038738  0.251950
      B -0.807841  0.254203
      C -0.015578 -0.273964
three A  0.091819       NaN
      B       NaN -1.780584
      C  0.142970       NaN
two   A       NaN -2.348998
      B -0.491300       NaN
      C       NaN -1.037244

In [111]:
pd.pivot_table(df, values=['D','E'], index=['B'], columns=['A', 'C'], aggfunc=np.sum)

D                                                           E  \
A       one               three                 two                 one   
C       bar       foo       bar       foo       bar       foo       bar   
B                                                                         
A  0.077477  0.503900  0.183637       NaN       NaN -4.697996  2.609344   
B -1.615682  0.508406       NaN -3.561168 -0.982599       NaN  0.979810   
C -0.031155 -0.547928  0.285941       NaN       NaN -2.074487  0.608597   

                                                   
A               three               two            
C       foo       bar     foo       bar       foo  
B                                                  
A  1.250415 -0.826732     NaN       NaN -2.357232  
B  0.885692       NaN  1.1826 -0.812608       NaN  
C  0.017308  0.603348     NaN       NaN  0.209642

In [114]:
pd.pivot_table(df, index=['A', 'B'], columns=['C'],aggfunc='sum',fill_value=0)

D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A  0.077477  0.503900  2.609344  1.250415
      B -1.615682  0.508406  0.979810  0.885692
      C -0.031155 -0.547928  0.608597  0.017308
three A  0.183637  0.000000 -0.826732  0.000000
      B  0.000000 -3.561168  0.000000  1.182600
      C  0.285941  0.000000  0.603348  0.000000
two   A  0.000000 -4.697996  0.000000 -2.357232
      B -0.982599  0.000000 -0.812608  0.000000
      C  0.000000 -2.074487  0.000000  0.209642

In [117]:
pd.pivot_table(df,index=pd.Grouper(freq='3M',key='F'),values='D',columns='B')

'B                  A         B         C\nF                                       \n2013-01-31  0.251950        No        No\n2013-04-30  0.091819  0.254203 -1.037244\n2013-07-31 -2.348998 -0.807841 -0.015578\n2013-10-31  0.038738 -1.780584 -0.273964\n2014-01-31        No -0.491300  0.142970'

In [118]:
# margins
# crosstab
df = pd.DataFrame({'A': [1, 2, 2, 2, 2], 'B': [3, 3, 4, 4, 4],
                      'C': [1, 1, np.nan, 1, 1]})

In [119]:
df

,A,B,C
0,1,3,1.0
1,2,3,1.0
2,2,4,NaN
3,2,4,1.0
4,2,4,1.0


In [120]:
pd.crosstab(df.A, df.B)

B,3,4
A,,
1,1,0
2,1,3


In [121]:
pd.crosstab(df.A,df.B,normalize=True)

B,3,4
A,,
1,0.2,0.0
2,0.2,0.6


In [122]:
pd.crosstab(df.A,df.B,normalize='index')

B,3,4
A,,
1,1.00,0.00
2,0.25,0.75


In [125]:
pd.crosstab(df.A,df.B,margins=True)

B,3,4,All
A,,,
1,1,0,1
2,1,3,4
All,2,3,5


In [126]:
df = pd.DataFrame({'key': list('bbacab'), 'data1': range(6)})

In [127]:
df

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [128]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [136]:
values = np.random.randn(10)
bins = np.arange(-1,1,0.2)

pd.get_dummies(pd.cut(values, bins))


,"(-1.0, -0.8]","(-0.8, -0.6]","(-0.6, -0.4]","(-0.4, -0.2]","(-0.2, -2.22e-16]","(-2.22e-16, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]"
0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,1,0,0,0
7,0,0,0,0,0,0,0,0,0
8,0,0,1,0,0,0,0,0,0
9,0,0,0,0,0,0,0,1,0


In [130]:
values

array([-0.39004214, -0.68311842,  0.60356288,  2.15847084, -0.78908937,
        0.41151979, -2.5448007 ,  0.02076287, -0.39181511,  0.77494727])

In [135]:
np.arange(-1,1,0.2)

array([ -1.00000000e+00,  -8.00000000e-01,  -6.00000000e-01,
        -4.00000000e-01,  -2.00000000e-01,  -2.22044605e-16,
         2.00000000e-01,   4.00000000e-01,   6.00000000e-01,
         8.00000000e-01])

In [137]:
df = pd.DataFrame({'A': ['a', 'b', 'a'], 'B': ['c', 'c', 'b'],
                    'C': [1, 2, 3]})

In [138]:
pd.get_dummies(df)

,C,A_a,A_b,B_b,B_c
0,1,1,0,0,1
1,2,0,1,0,1
2,3,1,0,1,0


In [140]:
df.drop('B',axis=1)

,A,C
0,a,1
1,b,2
2,a,3
